In [ ]:
from matplotlib import pyplot as plt
from collections import Counter, defaultdict
import seaborn as sns
import pandas as pd
import networkx as nx
import psycopg2
import pickle
import os
import re

import warnings

warnings.filterwarnings("ignore")

%load_ext nb_black

In [ ]:
def connect():
    conn = psycopg2.connect(
        user="postgres",
        password=os.environ.get("POSTGRES_PASS", ""),
        host="localhost",
        port=5432,
        database="venmo",
    )
    return conn

In [ ]:
DG = nx.DiGraph()
meta = Counter()

conn = connect()
with conn.cursor(name="network_x") as cursor:
    cursor.itersize = 2000
    cursor.execute("SELECT * FROM transactions")
    for i, row in enumerate(cursor):
        meta["msgs"] += 1
        if i % 2_000_000 == 0 and i != 0:
            # checkpoint
            print("Checkpoint", i)
            with open("graph_ckpt.pkl", "wb") as f:
                pickle.dump((meta, DG), f)
        from_id = row[6]
        to_id = row[7]
        DG.add_edge(from_id, to_id)

conn.close()

In [ ]:
# Checkpoint
with open("graph_ckpt.pkl", "rb") as f:
    meta_saved, DG_saved = pickle.load(f)